# Progetto MCS

Per la gestione della struttura dati e le operazioni elementari fra matrici è richiesto di partire da una libreria open-source, come Eigen, Armadillo, blas/lapack. Oppure, qualora il linguaggio di programmazione lo permetta utilizzare vettori e matrici già implementate al suo interno.


## Data Import

In [1]:
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix, csc_matrix, tril
from scipy.sparse.linalg import spsolve_triangular

In [3]:
data = {
    "spa1" : 0,
    "spa2" : 0,
    "vem1" : 0,
    "vem2" : 0}
for x in data:
    data[x] = {
        "A" : mmread("data/" + x + ".mtx"),
        "x" : 0,
        "b" : 0,
    }
    data[x]["A"] = csr_matrix(data[x]["A"])
    data[x]["x"] = np.array([1.0]*data[x]["A"].get_shape()[0])
    data[x]["b"] = np.array(data[x]["A"].dot(data[x]["x"]))
tols = [10**(-4), 10**(-6), 10**(-8), 10**(-10)]

# TODO
### - Cosa che resituisca i tempi delle singole esecuzioni
### - Errore assoluto (se serve)
### - Grafici
### - Metodo preliminare per i metodi che lo hanno (es. jacobi che dice subito se la matrice è risolvibile o meno )
### - Fare file di python eseguibile
### - Sistemare e capire come raggruppare il codice

In [ ]:
#jacobi
from jacoby_mcs import Jacoby
ja = Jacoby()
files_jacobi = {}
for file in data:
    jacobi_results=[]
    for tol in tols:
        jacobi_results.append(ja.execute(data[file]["A"], data[file]["b"], data[file]["x"], tol))
    files_jacobi[file] = jacobi_results

In [ ]:
files_jacobi

In [ ]:
#gradiente_coniugato
from gradiente_coniugato import GradC
gc = GradC()
files_gradC = {}
for file in data:
    gradC_results=[]
    for tol in tols:
        gradC_results.append(gc.execute(data[file]["A"], data[file]["b"], data[file]["x"], tol))
    files_gradC[file] = gradC_results

In [ ]:
files_gradC

Condizione di arresto: $\frac{||Ax^{(k)}-b||}{||b||}<tol$

Reminder: 

1.   **Iniziare le iterazioni con il vettore x nullo**
2.   **tol = [$10^{-4}, 10^{-6}, 10^{-8}, 10^{-10}]$**
3.   **Dichiarare di non essere giunti a convergenza se k > maxiter dove maxiter lo scegliamo (>= 20000)**




## Metodo di Jacobi
https://en.wikipedia.org/wiki/Jacobi_method

In [ ]:
niter = 0
new_vector = np.zeros(spa1.get_shape()[0])
inverted_p_matrix = inv(np.diagflat(spa1.diagonal()))
tol = 1e-4
while True:
    residual = spa1b - spa1.dot(new_vector)
    new_vector = new_vector + (inverted_p_matrix.dot(residual))
    niter = niter +1
    if(np.linalg.norm(residual, ord=2)/np.linalg.norm(spa1b, ord=2) < tol or niter >= 20000):
      break;
print(niter)
print(new_vector)

In [ ]:
#jacobi secondo slide del prof
niter = 0
new_vector = np.asarray([0]*len(spa1x))
inverted_p_matrix = 1/spa1.diagonal()
tol = 10**(-4)
residual = spa1b - spa1.dot(new_vector)
while np.linalg.norm(residual)/np.linalg.norm(spa1b) >= tol and niter <= 20000:
    new_vector = new_vector + (inverted_p_matrix * (residual))
    residual = spa1b - spa1.dot(new_vector)
    niter = niter +1
    print(np.linalg.norm(residual)/np.linalg.norm(spa1b))

### Errore relativo - numero iterazioni - tempo di calcolo

In [ ]:
#numero iterazioni
niter

In [ ]:
#errore relativo
(np.linalg.norm(np.subtract(new_vector, spa1x))/np.linalg.norm(spa1x))*100

## Metodo di Gauß-Seidel

In [4]:
## Parametri
mtxA = data["vem2"]["A"].copy()
mtxP = tril(mtxA, format="csr")
max_iter = 20000
vectB = data["vem2"]["b"].copy()
tol = tols[3]

## Variabili
k = 0
vectX = np.zeros(mtxA.shape[0])
residual = vectB - mtxA.dot(vectX)

## Funzione
while np.linalg.norm(residual)/np.linalg.norm(vectB) >= tol and k < max_iter:
    k += 1
    vectX = vectX + spsolve_triangular(mtxP,residual, lower = True)
    residual = vectB - mtxA.dot(vectX)
    
    ## Stampe di controllo
    if k%100 == 0:
        print(k, end = "\t")
        if k%1000 == 0:
            print()
            print(np.linalg.norm(residual))
            print(np.linalg.norm(residual)/np.linalg.norm(vectB))
print(k)
print(np.linalg.norm(residual))
print(np.linalg.norm(residual)/np.linalg.norm(vectB))
print(vectX)

100	200	300	400	500	600	700	800	900	1000	
0.0016603854755805065
8.29933424106669e-05
1100	1200	1300	1400	1500	1600	1700	1800	1900	2000	
8.58123796102091e-06
4.289278790260593e-07
2100	2200	2300	2400	2500	2600	2700	2800	2900	3000	
4.4349387950950427e-08
2.2167767630163595e-09
3100	3200	3300	3400	3500	3589
1.995462062625758e-09
9.97419386441539e-11
[1. 1. 1. ... 1. 1. 1.]


In [ ]:
import gauss_seidel as gs

for el in data:
    for tol in tols:
        gs.solve(mtxA=data[el]["A"], vectB=data[el]["b"], tol=tol, maxIter=20000)

## Metodo del Gradiente


1. $r^{(k)} = b -Ax^{(k)}$
2. $y^{(k)} = Ar^{(k)}$
3. $a = (r^{(k)})^tr^{(k)}$
4. $b = (r^{(k)})^ty^{(k)}$
5. $\alpha_k = a/b$
6. $x^{(k+1)} = r^{(k)} \alpha_kr^{(k)}$

In [7]:
# Parametri
mtxA = data["spa1"]["A"].copy()
max_iter = 20000
vectB = data["spa1"]["b"].copy()
tol = tols[0]

# Variabili
k = 0
vectX = np.zeros(mtxA.shape[0])
residual = vectB - mtxA.dot(vectX)

# Funzione
while np.linalg.norm(residual)/np.linalg.norm(vectB) >= tol and k < max_iter:
    k += 1

    y = mtxA.dot(residual)
    a = residual.T.dot(residual)
    b = residual.T.dot(y)
    alpha = a/b

    vectX = vectX + alpha * residual

    ## Compressed version
    #vectX = vectX + (residual.T.dot(residual) / residual.T.dot(mtxA.dot(residual))) * residual

    residual = vectB - mtxA.dot(vectX)

    # Stampe di controllo
    if k%100 == 0:
        print(k, end = "\t")
        if k%1000 == 0:
            print()
            print(np.linalg.norm(residual))
            print(np.linalg.norm(residual)/np.linalg.norm(vectB))
print(k)
print(np.linalg.norm(residual))
print(np.linalg.norm(residual)/np.linalg.norm(vectB))
print(vectX)


100	143
2.0473746617711672
9.909686562570107e-05
[1.00016623 1.0001564  1.00019454 1.00091519 1.00039395 0.99992446
 1.00086744 1.0002122  0.99985525 1.00013945 0.99975745 1.00053372
 1.0001407  1.0004963  0.99993207 1.00004995 0.99996522 0.99960646
 1.00001873 1.0025082  1.00013105 1.01137004 1.00168806 1.00539522
 1.00097526 1.00007659 1.00029358 0.99998973 0.99799755 1.00246547
 0.99983946 0.98094735 0.99989156 1.00148665 1.00009169 0.9997569
 0.99984547 0.99991695 1.00086849 1.00005044 0.99995613 1.00054577
 1.00101057 1.0006179  0.9997192  1.00189249 1.00048689 0.99972297
 0.99799295 1.00016718 0.9998889  1.00011761 1.00007132 1.00055613
 1.00156459 1.0008735  1.00012657 1.0262186  1.00006944 1.0006255
 1.00022654 1.00111278 0.99993583 0.99988368 1.0034584  0.9999687
 1.0001616  0.99981575 1.00040744 0.99865927 1.00038913 1.00042138
 1.00017385 1.00096636 0.99982206 0.99983294 1.00238143 1.00046563
 0.99958567 1.0002059  0.99988755 0.99992315 1.00023411 1.00037418
 0.99985022 1.00

## Metodo del Gradiente coniugato

- Un vettore ottimale rispetto a una direzione d se d*r(k)=0
- x(k+1) è ottimale rispetto a r(k+1)
- x(k+1) = x(k) + a(k)d(k)
- a(k) = ( d(k)^t * r(k) ) / ( d(k)^t * Ad(k) )
- d(k+1) = r(k+1) - b(k)*d(k)
- b(k) = ( d(k)^t * Ar(k+1) ) / ( d(k)^t * Ad(k) )



In [ ]:
#gradiente coniugato
#capire se in ak e bk bisogna calcolare la derivata di dir
niter = 0
new_vector = np.asarray([0]*len(spa1x))
tol = 10**(-4)
residual = spa1b - spa1.dot(new_vector)
dir = residual.copy()
while np.linalg.norm(residual)/np.linalg.norm(spa1b) >= tol and niter <= 20000:
    #r = spa1b - spa1.dot(new_vector)
    y = spa1.dot(dir)
    z = spa1.dot(residual)
    ak = (np.dot(dir, residual)) / (np.dot(dir, y))
    new_vector = new_vector + (ak * dir)
    residual = spa1b - spa1.dot(new_vector)
    w = spa1.dot(residual)
    bk = (np.dot(dir, w)) / (np.dot(dir, y))
    dir = residual - (bk*dir)

    residual = spa1b - spa1.dot(new_vector)
    niter = niter + 1

In [ ]:
niter

In [ ]:
np.linalg.norm(np.subtract(new_vector, spa1x))/np.linalg.norm(spa1b)